In [1]:
import mysql.connector
import decimal
import pprint
import csv
from datetime import datetime

In [2]:
con = mysql.connector.connect(host="#######", 
                              user="#######", passwd="#######", db="#######")
cursor = con.cursor(buffered=True)
ids_teste = "(1,2,322,333,348,353,3086,12402,17701,17711,123325,123406,123417,123349,123301,123310,123401,123371,123375,123314,123347,123356)"

In [3]:
pp = pprint.PrettyPrinter(indent=4)
#COLOCAR AQUI O MES E ANO CORRESPONDENTE
mes = 3
ano = 2022

#CLIENTES ATIVOS COM CONSUMO NO MES
relatorio = {}
total_clientes = 0
total_fc_analise_mais_qtd = 0
total_fc_analise_mais_valor = 0
total_fc_analise_qtd = 0
total_fc_analise_valor = 0
total_fc_empresa_qtd = 0
total_fc_empresa_valor = 0
total_fc_renda_qtd = 0
total_fc_renda_valor = 0
total_total_qtd = 0
total_total_valor = 0
total_locacoes = 0

query = "select p.id, p.nome, ac.condicao, pp.produto_id, pp.valor, pp.payment_at, s.id, date(p.created_at), p.status "
query += "from pretendente_produto pp "
query += "inner join pretendente pr on pr.id = pp.pretendente_id "
query += "inner join solicitacao s on pr.solicitacao_id = s.id "
query += "inner join pessoa p on p.id = s.cliente_id "
query += "inner join administradora_cliente ac on p.id = ac.id "
query += "where month(pp.payment_at) = "+str(mes)+" and year(pp.payment_at) = "+str(ano)+" "
query += "and p.status in ('ATIVO', 'INATIVO', 'SUSPENSO', 'BLOQUEADO') "
query += "and p.id not in "+ids_teste+" "
query += "order by p.nome"

cursor.execute(query)
for resultado in cursor:
    if resultado[0] not in relatorio:
        relatorio[resultado[0]] = {}
        relatorio[resultado[0]]['nome_cliente'] = resultado[1]
        relatorio[resultado[0]]['data_cadastro'] = resultado[7]
        relatorio[resultado[0]]['status'] = resultado[8]
        relatorio[resultado[0]]['pagamento'] = resultado[2]
        relatorio[resultado[0]]['fc analise+ qtd'] = 0
        relatorio[resultado[0]]['fc analise+ valor'] = 0
        relatorio[resultado[0]]['fc analise qtd'] = 0
        relatorio[resultado[0]]['fc analise valor'] = 0
        relatorio[resultado[0]]['fc empresa qtd'] = 0
        relatorio[resultado[0]]['fc empresa valor'] = 0
        relatorio[resultado[0]]['fc renda qtd'] = 0
        relatorio[resultado[0]]['fc renda valor'] = 0
        relatorio[resultado[0]]['total qtd'] = 0
        relatorio[resultado[0]]['total valor'] = 0
        relatorio[resultado[0]]['locacoes'] = []
    if resultado[3] == 1:
        relatorio[resultado[0]]['fc analise+ qtd'] += 1
        relatorio[resultado[0]]['fc analise+ valor'] += resultado[4]
        total_fc_analise_mais_qtd += 1
        total_fc_analise_mais_valor += resultado[4]
    if resultado[3] == 3:
        relatorio[resultado[0]]['fc analise qtd'] += 1
        relatorio[resultado[0]]['fc analise valor'] += resultado[4]
        total_fc_analise_qtd += 1
        total_fc_analise_valor += resultado[4]
    if resultado[3] == 4:
        relatorio[resultado[0]]['fc empresa qtd'] += 1
        relatorio[resultado[0]]['fc empresa valor'] += resultado[4]
        total_fc_empresa_qtd += 1
        total_fc_empresa_valor += resultado[4]
    if resultado[3] == 2:
        relatorio[resultado[0]]['fc renda qtd'] += 1
        relatorio[resultado[0]]['fc renda valor'] += resultado[4]
        total_fc_renda_qtd += 1
        total_fc_renda_valor += resultado[4]
    relatorio[resultado[0]]['total qtd'] += 1
    total_total_qtd += 1
    relatorio[resultado[0]]['total valor'] += resultado[4]
    total_total_valor += resultado[4]
    if resultado[6] not in relatorio[resultado[0]]['locacoes']:
        relatorio[resultado[0]]['locacoes'].append(resultado[6])
        total_locacoes += 1

In [4]:
#RESTO DOS CLIENTES (CLIENTES ATIVOS QUE NAO CONSUMIRAM NO MÊS)
relatorio_resto_clientes = {}
total_resto_clientes_clientes = 0
total_resto_clientes_fc_analise_mais_qtd = 0
total_resto_clientes_fc_analise_mais_valor = 0
total_resto_clientes_fc_analise_qtd = 0
total_resto_clientes_fc_analise_valor = 0
total_resto_clientes_fc_empresa_qtd = 0
total_resto_clientes_fc_empresa_valor = 0
total_resto_clientes_fc_renda_qtd = 0
total_resto_clientes_fc_renda_valor = 0
total_resto_clientes_total_qtd = 0
total_resto_clientes_total_valor = 0
total_resto_clientes_locacoes = 0

query = "select p.id, p.nome, date(p.created_at), ac.condicao, p.status "
query += "from pessoa p "
query += "inner join administradora_cliente ac on ac.id = p.id "
query += "where p.id not in "+ids_teste+" "
query += "and p.status = 'ATIVO' "
query += "and p.id not in "
query += "("
query += "select p.id "
query += "from pretendente_produto pp "
query += "inner join pretendente pr on pr.id = pp.pretendente_id "
query += "inner join solicitacao s on pr.solicitacao_id = s.id "
query += "inner join pessoa p on p.id = s.cliente_id "
query += "inner join administradora_cliente ac on p.id = ac.id "
query += "where month(pp.payment_at) = "+str(mes)+" and year(pp.payment_at) = "+str(ano)+" "
query += ") "
query += "order by p.nome "

cursor.execute(query)
for resultado in cursor:
    if resultado[0] not in relatorio_resto_clientes:
        relatorio_resto_clientes[resultado[0]] = {}
        relatorio_resto_clientes[resultado[0]]['nome_cliente'] = resultado[1]
        relatorio_resto_clientes[resultado[0]]['data_cadastro'] = resultado[2]
        relatorio_resto_clientes[resultado[0]]['status'] = resultado[4]
        relatorio_resto_clientes[resultado[0]]['pagamento'] = resultado[3]
        relatorio_resto_clientes[resultado[0]]['fc analise+ qtd'] = 0
        relatorio_resto_clientes[resultado[0]]['fc analise+ valor'] = 0
        relatorio_resto_clientes[resultado[0]]['fc analise qtd'] = 0
        relatorio_resto_clientes[resultado[0]]['fc analise valor'] = 0
        relatorio_resto_clientes[resultado[0]]['fc empresa qtd'] = 0
        relatorio_resto_clientes[resultado[0]]['fc empresa valor'] = 0
        relatorio_resto_clientes[resultado[0]]['fc renda qtd'] = 0
        relatorio_resto_clientes[resultado[0]]['fc renda valor'] = 0
        relatorio_resto_clientes[resultado[0]]['total qtd'] = 0
        relatorio_resto_clientes[resultado[0]]['total valor'] = 0
        relatorio_resto_clientes[resultado[0]]['locacoes'] = []

In [5]:
#CLIENTES TESTE
relatorio_teste = {}
total_teste_clientes = 0
total_teste_fc_analise_mais_qtd = 0
total_teste_fc_analise_mais_valor = 0
total_teste_fc_analise_qtd = 0
total_teste_fc_analise_valor = 0
total_teste_fc_empresa_qtd = 0
total_teste_fc_empresa_valor = 0
total_teste_fc_renda_qtd = 0
total_teste_fc_renda_valor = 0
total_teste_total_qtd = 0
total_teste_total_valor = 0
total_teste_locacoes = 0

query = "select p.id, p.nome, ac.condicao, pp.produto_id, pp.valor, pp.payment_at, s.id, date(p.created_at), p.status "
query += "from pretendente_produto pp "
query += "inner join pretendente pr on pr.id = pp.pretendente_id "
query += "inner join solicitacao s on pr.solicitacao_id = s.id "
query += "inner join pessoa p on p.id = s.cliente_id "
query += "inner join administradora_cliente ac on p.id = ac.id "
query += "where month(pp.payment_at) = "+str(mes)+" and year(pp.payment_at) = "+str(ano)+" "
query += "and p.id in "+ids_teste+" "
query += "order by p.nome"

cursor.execute(query)
for resultado in cursor:
    if resultado[0] not in relatorio_teste:
        relatorio_teste[resultado[0]] = {}
        relatorio_teste[resultado[0]]['nome_cliente'] = resultado[1]
        relatorio_teste[resultado[0]]['data_cadastro'] = resultado[7]
        relatorio_teste[resultado[0]]['status'] = resultado[8]
        relatorio_teste[resultado[0]]['pagamento'] = resultado[2]
        relatorio_teste[resultado[0]]['fc analise+ qtd'] = 0
        relatorio_teste[resultado[0]]['fc analise+ valor'] = 0
        relatorio_teste[resultado[0]]['fc analise qtd'] = 0
        relatorio_teste[resultado[0]]['fc analise valor'] = 0
        relatorio_teste[resultado[0]]['fc empresa qtd'] = 0
        relatorio_teste[resultado[0]]['fc empresa valor'] = 0
        relatorio_teste[resultado[0]]['fc renda qtd'] = 0
        relatorio_teste[resultado[0]]['fc renda valor'] = 0
        relatorio_teste[resultado[0]]['total qtd'] = 0
        relatorio_teste[resultado[0]]['total valor'] = 0
        relatorio_teste[resultado[0]]['locacoes'] = []
    if resultado[3] == 1:
        relatorio_teste[resultado[0]]['fc analise+ qtd'] += 1
        relatorio_teste[resultado[0]]['fc analise+ valor'] += resultado[4]
        total_teste_fc_analise_mais_qtd += 1
        total_teste_fc_analise_mais_valor += resultado[4]
    if resultado[3] == 3:
        relatorio_teste[resultado[0]]['fc analise qtd'] += 1
        relatorio_teste[resultado[0]]['fc analise valor'] += resultado[4]
        total_teste_fc_analise_qtd += 1
        total_teste_fc_analise_valor += resultado[4]
    if resultado[3] == 4:
        relatorio_teste[resultado[0]]['fc empresa qtd'] += 1
        relatorio_teste[resultado[0]]['fc empresa valor'] += resultado[4]
        total_teste_fc_empresa_qtd += 1
        total_teste_fc_empresa_valor += resultado[4]
    if resultado[3] == 2:
        relatorio_teste[resultado[0]]['fc renda qtd'] += 1
        relatorio_teste[resultado[0]]['fc renda valor'] += resultado[4]
        total_teste_fc_renda_qtd += 1
        total_teste_fc_renda_valor += resultado[4]
    relatorio_teste[resultado[0]]['total qtd'] += 1
    total_teste_total_qtd += 1
    relatorio_teste[resultado[0]]['total valor'] += resultado[4]
    total_teste_total_valor += resultado[4]
    if resultado[6] not in relatorio_teste[resultado[0]]['locacoes']:
        relatorio_teste[resultado[0]]['locacoes'].append(resultado[6])
        total_teste_locacoes += 1

In [6]:
relatorio1 = relatorio
relatorio2 = relatorio_resto_clientes
relatorio3 = relatorio_teste
csv_columns = ['ID', 'CLIENTE', 'DATA DE CADASTRO', 'STATUS', 'PAGAMENTO', 'FC ANALISE+ QTD', 'FC ANALISE+ VALOR', 
               'FC ANALISE QTD', 'FC ANALISE VALOR', 'FC EMPRESA QTD', 'FC EMPRESA VALOR', 'FC RENDA QTD',
               'FC RENDA VALOR', 'TOTAL QTD', 'TOTAL VALOR', 'LOCACOES']

with open('relatorio_producao_cobranca_'+str(mes)+'_'+str(ano)+'.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';')
    spamwriter.writerow(csv_columns)
    for key in relatorio1:
        listaRow = list()
        listaRow.append(key)
        listaRow.append(relatorio1[key]['nome_cliente'])
        listaRow.append(relatorio1[key]['data_cadastro'])
        listaRow.append(relatorio1[key]['status'])
        listaRow.append(relatorio1[key]['pagamento'])
        listaRow.append(relatorio1[key]['fc analise+ qtd'])
        listaRow.append(relatorio1[key]['fc analise+ valor'])
        listaRow.append(relatorio1[key]['fc analise qtd'])
        listaRow.append(relatorio1[key]['fc analise valor'])
        listaRow.append(relatorio1[key]['fc empresa qtd'])
        listaRow.append(relatorio1[key]['fc empresa valor'])
        listaRow.append(relatorio1[key]['fc renda qtd'])
        listaRow.append(relatorio1[key]['fc renda valor'])
        listaRow.append(relatorio1[key]['total qtd'])
        listaRow.append(relatorio1[key]['total valor'])
        listaRow.append(len(relatorio1[key]['locacoes']))

        spamwriter.writerow(listaRow)
    spamwriter.writerow(list())
    
    for key in relatorio2:
        listaRow = list()
        listaRow.append(key)
        listaRow.append(relatorio2[key]['nome_cliente'])
        listaRow.append(relatorio2[key]['data_cadastro'])
        listaRow.append(relatorio2[key]['status'])
        listaRow.append(relatorio2[key]['pagamento'])
        listaRow.append(relatorio2[key]['fc analise+ qtd'])
        listaRow.append(relatorio2[key]['fc analise+ valor'])
        listaRow.append(relatorio2[key]['fc analise qtd'])
        listaRow.append(relatorio2[key]['fc analise valor'])
        listaRow.append(relatorio2[key]['fc empresa qtd'])
        listaRow.append(relatorio2[key]['fc empresa valor'])
        listaRow.append(relatorio2[key]['fc renda qtd'])
        listaRow.append(relatorio2[key]['fc renda valor'])
        listaRow.append(relatorio2[key]['total qtd'])
        listaRow.append(relatorio2[key]['total valor'])
        listaRow.append(len(relatorio2[key]['locacoes']))
        
        spamwriter.writerow(listaRow)
    
    spamwriter.writerow(list())
    listaTotais = list()
    listaTotais.append('TOTAL')
    listaTotais.append(len(relatorio1)+len(relatorio2))
    listaTotais.append('-')
    listaTotais.append('-')
    listaTotais.append('-')
    listaTotais.append(total_fc_analise_mais_qtd)
    listaTotais.append(total_fc_analise_mais_valor)
    listaTotais.append(total_fc_analise_qtd)
    listaTotais.append(total_fc_analise_valor)
    listaTotais.append(total_fc_empresa_qtd)
    listaTotais.append(total_fc_empresa_valor)
    listaTotais.append(total_fc_renda_qtd)
    listaTotais.append(total_fc_renda_valor)
    listaTotais.append(total_total_qtd)
    listaTotais.append(total_total_valor)
    listaTotais.append(total_locacoes)
    spamwriter.writerow(listaTotais)
    
    spamwriter.writerow(list())
    spamwriter.writerow(list())
    
    spamwriter.writerow(csv_columns)
    for key in relatorio3:
        listaRow = list()
        listaRow.append(key)
        listaRow.append(relatorio3[key]['nome_cliente'])
        listaRow.append(relatorio3[key]['data_cadastro'])
        listaRow.append(relatorio3[key]['status'])
        listaRow.append(relatorio3[key]['pagamento'])
        listaRow.append(relatorio3[key]['fc analise+ qtd'])
        listaRow.append(relatorio3[key]['fc analise+ valor'])
        listaRow.append(relatorio3[key]['fc analise qtd'])
        listaRow.append(relatorio3[key]['fc analise valor'])
        listaRow.append(relatorio3[key]['fc empresa qtd'])
        listaRow.append(relatorio3[key]['fc empresa valor'])
        listaRow.append(relatorio3[key]['fc renda qtd'])
        listaRow.append(relatorio3[key]['fc renda valor'])
        listaRow.append(relatorio3[key]['total qtd'])
        listaRow.append(relatorio3[key]['total valor'])
        listaRow.append(len(relatorio3[key]['locacoes']))

        spamwriter.writerow(listaRow)
    listaTotais = list()
    listaTotais.append('TOTAL')
    listaTotais.append(len(relatorio3))
    listaTotais.append('-')
    listaTotais.append('-')
    listaTotais.append('-')
    listaTotais.append(total_teste_fc_analise_mais_qtd)
    listaTotais.append(total_teste_fc_analise_mais_valor)
    listaTotais.append(total_teste_fc_analise_qtd)
    listaTotais.append(total_teste_fc_analise_valor)
    listaTotais.append(total_teste_fc_empresa_qtd)
    listaTotais.append(total_teste_fc_empresa_valor)
    listaTotais.append(total_teste_fc_renda_qtd)
    listaTotais.append(total_teste_fc_renda_valor)
    listaTotais.append(total_teste_total_qtd)
    listaTotais.append(total_teste_total_valor)
    listaTotais.append(total_teste_locacoes)
    spamwriter.writerow(listaTotais)
    
        
        
print('Script executado com sucesso!')

Script executado com sucesso!
